## Linear Regression

In [ ]:
import math
import pymc as pm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

We can generate an example dataset by adding noise to a simple linear relationship.

In [ ]:
N = 10

c = 3
m = 2

x = np.linspace(1, N, N)
y = m * x + c + np.random.normal(size=N)

A plot of y against x looks roughly linear.

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.xlabel('x')
plt.ylabel('y');

We might use a normal least-squares regression package to find the best fit line.

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x.reshape(-1, 1), y)

x_predict = np.linspace(0, 10, 1001)
y_predict = reg.coef_[0] * x_predict + reg.intercept_

plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
reg.intercept_

In [ ]:
reg.coef_[0]

We can describe the same model with probablistic programming.

In [ ]:
model = pm.Model() 

with model:
    
    m = pm.Normal('m', mu=0.0, sigma=1.0)

    c = pm.Normal('c', mu=0.0, sigma=1.0)
    
    sd = pm.Exponential('sd', lam=1.0)

    mu = m * x + c

    obs = pm.ConstantData('obs', y)
    
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sd, observed=obs)

In [ ]:
with model:
    
    idata = pm.sample(draws=2000)

In [ ]:
with model:
  
  pm.plot_trace(idata, var_names=['m', 'c', 'sd'], combined=False);

In [ ]:
m = np.mean(np.asarray(idata.posterior.get('m')[0]))
c = np.mean(np.asarray(idata.posterior.get('c')[0]))

x_predict = np.linspace(0, 10, 1001)
y_predict = m * x_predict + c

plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
sns.kdeplot(x=idata.posterior.get('m')[0], y=idata.posterior.get('c')[0])
plt.xlabel('m')
plt.ylabel('c')

In [ ]:
m = np.asarray(idata.posterior.get('m')[0])
c = np.asarray(idata.posterior.get('c')[0])

plt.figure(figsize=(10, 4))
for i in range(1000):
  x_predict = np.linspace(0, 10, 1001)
  y_predict = m[i] * x_predict + c[i]
  plt.plot(x_predict, y_predict, 'r', alpha=0.01);
plt.plot(x, y, 'o');
plt.xlabel('x')
plt.ylabel('y');

## Handling Outliers


If we have outliers in our dataset, standard linear regression is a bit more tricky.

In [ ]:
y_outlier = y

y_outlier[3] += 20

If we apply linear regression the single outlier affects our estimate of the gradient and intercept.

In [ ]:
reg = LinearRegression().fit(x.reshape(-1, 1), y_outlier)

x_predict = np.linspace(0, 10, 1001)
y_predict = reg.coef_[0] * x_predict + reg.intercept_

plt.figure(figsize=(10, 4))
plt.plot(x, y_outlier, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

However, we can build the existance of outliers into our probablistic model.

In [ ]:
model = pm.Model() 

with model:
    
    m = pm.Normal('m', mu=0.0, sigma=1.0)

    c = pm.Normal('c', mu=0.0, sigma=1.0)
    
    sd_data = pm.Exponential('sd', lam=1.0)

    sd_outlier = 100

    P_outlier = pm.Bernoulli('outlier', p=0.1, shape = (N))

    sd = pm.math.switch(P_outlier, sd_outlier, sd_data)
    
    mu = m * x + c

    obs = pm.ConstantData('obs', y_outlier)

    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sd, observed=obs)

In [ ]:
with model:
    
    idata = pm.sample(draws=2000)

Taking the mean of the posterior and ploting the resulting straight line shows the effectiveness of this approach.

In [ ]:
m = np.mean(np.asarray(idata.posterior.get('m')[0]))
c = np.mean(np.asarray(idata.posterior.get('c')[0]))

x_predict = np.linspace(0, 10, 1001)
y_predict = m * x_predict + c

plt.figure(figsize=(10, 4))
plt.plot(x, y_outlier, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

We can use the trace to calculate the probability that any point is an outlier.

In [ ]:
prob = np.asarray(idata.posterior.get('outlier')[0])

plt.bar(x, np.mean(prob, 0))
plt.xlabel('x')
plt.ylabel('P_outlier')